In [1]:
import numpy as np
import pandas as pd
import pyreadr
import os
import hickle as hkl
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
import heapq
import matplotlib.pyplot as plt
import math
from matplotlib.font_manager import FontProperties
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [2]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [3]:
#darmanis
romanov_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_feature.csv').iloc[:,1].values.tolist()
zeisel_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_feature.csv').iloc[:,1].values.tolist()
darmanis_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_feature.csv').iloc[:,1].values.tolist()

In [4]:
romanov_feature = [i.upper() for  i in romanov_feature]
zeisel_feature = [i.upper() for  i in zeisel_feature]

In [5]:
romanov_zeisel = intersection(romanov_feature, zeisel_feature)
romanov_zeisel_darmanis =  intersection(romanov_zeisel,darmanis_feature)
romanov_feature_share = [romanov_feature.index(i) for i in romanov_zeisel_darmanis]
zeisel_feature_share = [zeisel_feature.index(i) for i in romanov_zeisel_darmanis]
darmanis_feature_share = [darmanis_feature.index(i) for i in romanov_zeisel_darmanis]

In [6]:
len(romanov_feature)

24341

In [90]:
print(romanov_logcounts.shape)
print(zeisel_logcounts.shape)
print(darmanis_logcounts.shape)
test_name

(24341, 2881)
(19972, 3005)
(22088, 285)


'segerstolpe'

In [7]:
romanov_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_logcounts.csv').iloc[:,1:].values
zeisel_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_logcounts.csv').iloc[:,1:].values
darmanis_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_logcounts.csv').iloc[:,1:].values
romanov_logcounts_share = romanov_logcounts[romanov_feature_share, :]
zeisel_logcounts_share = zeisel_logcounts[zeisel_feature_share, :]
darmanis_logcounts_share = darmanis_logcounts[darmanis_feature_share, :]
feature_number = len(romanov_zeisel_darmanis)
romanov_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_label.csv').iloc[:,1].values.tolist()
zeisel_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_label.csv').iloc[:,1].values.tolist()
darmanis_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_label.csv').iloc[:,1].values.tolist()

In [91]:
for i in range(len(romanov_label)):
    if romanov_label[i] == 'oligos':
        romanov_label[i] = 'oligodendrocytes'
for i in range(len(zeisel_label)):
    if zeisel_label[i] == 'ca1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 's1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 'interneurons':
        zeisel_label[i] = 'neurons'

In [92]:
pd.value_counts(romanov_label)

oligodendrocytes    1001
neurons              898
ependymal            356
astrocytes           267
endothelial          240
vsm                   71
microglia             48
dtype: int64

In [93]:
pd.value_counts(darmanis_label)

neurons             131
astrocytes           62
oligodendrocytes     38
endothelial          20
OPC                  18
microglia            16
dtype: int64

In [94]:
test_name = "darmanis"
data_train = np.hstack((romanov_logcounts_share, zeisel_logcounts_share))
test_logcounts_share = darmanis_logcounts_share
label_test = darmanis_label
label_train = romanov_label+zeisel_label
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [95]:
#PCA
data = np.hstack((romanov_logcounts_share,zeisel_logcounts_share,darmanis_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))

In [98]:
# 460147 论文方法
data_original = np.exp(romanov_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
romanov_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(zeisel_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
zeisel_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(darmanis_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
darmanis_scores = (data_train_var_log-y_pred).reshape(-1)

In [102]:
K = 1300
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_darmanis = intersection(ind_romanov_zeisel, ind_darmanis)
print(len(ind_romanov_zeisel_darmanis))
filtered_data_train = data_train[ind_romanov_zeisel_darmanis, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_460147.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_460147.hkl'%(test_name,K))

100


In [30]:
#myself 自己方法
zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(zeisel_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
zeisel_scores[feature_index_dict] = residuals
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(romanov_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
romanov_scores[feature_index_dict] = residuals
darmanis_scores = np.zeros(feature_number)-100
data_original = np.exp(darmanis_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(darmanis_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
darmanis_scores[feature_index_dict] = residuals

In [47]:
K = 641
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_darmanis = intersection(ind_romanov_zeisel, ind_darmanis)
print(len(ind_romanov_zeisel_darmanis))
filtered_data_train = data_train[ind_romanov_zeisel_darmanis, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))

100


In [103]:
#scmap
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
romanov_scores[feature_index_dict] = residuals
print(residuals.shape)

zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
zeisel_scores[feature_index_dict] = residuals
print(residuals.shape)

darmanis_scores = np.zeros(feature_number)-100
data_original = np.exp(darmanis_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
darmanis_scores[feature_index_dict] = residuals
print(residuals.shape)

(14492,)
(14651,)
(14487,)


In [108]:
K = 745
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_darmanis = intersection(ind_romanov_zeisel, ind_darmanis)
print(len(ind_romanov_zeisel_darmanis))
filtered_data_train = data_train[ind_romanov_zeisel_darmanis, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))

100


In [42]:
#superCT
print(data_train.shape)
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))

(14651, 5886)


In [ ]:
#romanov zeisel

In [49]:
romanov_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_feature.csv').iloc[:,1].values.tolist()
zeisel_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_feature.csv').iloc[:,1].values.tolist()
romanov_zeisel = intersection(romanov_feature, zeisel_feature)
romanov_feature_share = [romanov_feature.index(i) for i in romanov_zeisel]
zeisel_feature_share = [zeisel_feature.index(i) for i in romanov_zeisel]

In [50]:
romanov_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_logcounts.csv').iloc[:,1:].values
zeisel_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_logcounts.csv').iloc[:,1:].values
romanov_logcounts_share = romanov_logcounts[romanov_feature_share, :]
zeisel_logcounts_share = zeisel_logcounts[zeisel_feature_share, :]
feature_number = len(romanov_zeisel)
romanov_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_label.csv').iloc[:,1].values.tolist()
zeisel_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_label.csv').iloc[:,1].values.tolist()

In [52]:
pd.value_counts(romanov_label)

oligodendrocytes    1001
neurons              898
ependymal            356
astrocytes           267
endothelial          240
vsm                   71
microglia             48
dtype: int64

In [51]:
for i in range(len(romanov_label)):
    if romanov_label[i] == 'oligos':
        romanov_label[i] = 'oligodendrocytes'
for i in range(len(zeisel_label)):
    if zeisel_label[i] == 'ca1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 's1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 'interneurons':
        zeisel_label[i] = 'neurons'


In [53]:
pd.value_counts(zeisel_label)

neurons             1628
oligodendrocytes     820
astrocytes           198
endothelial          175
microglia             98
mural                 60
ependymal             26
dtype: int64

In [57]:
test_name = 'zeisel'
if test_name == 'zeisel':
    data_train = romanov_logcounts_share
    test_logcounts_share = zeisel_logcounts_share
    label_test = zeisel_label
    label_train = romanov_label
if test_name == 'romanov':
    data_train = zeisel_logcounts_share
    test_logcounts_share = romanov_logcounts_share
    label_test = romanov_label
    label_train = zeisel_label
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [28]:
#scmap
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
romanov_scores[feature_index_dict] = residuals
print(residuals.shape)

zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
zeisel_scores[feature_index_dict] = residuals
print(residuals.shape)

(19376,)
(19971,)


In [29]:
K = 149
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
print(len(ind_romanov_zeisel))
filtered_data_train = data_train[ind_romanov_zeisel, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))

100


In [30]:
#superCT
print(data_train.shape)
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))

(19971, 3005)


In [31]:
# 460147 论文方法
data_original = np.exp(romanov_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
romanov_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(zeisel_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
zeisel_scores = (data_train_var_log-y_pred).reshape(-1)

In [32]:
K = 210
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
print(len(ind_romanov_zeisel))
filtered_data_train = data_train[ind_romanov_zeisel, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_460147.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_460147.hkl'%(test_name,K))

100


In [58]:
#myself 自己方法
zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(zeisel_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
zeisel_scores[feature_index_dict] = residuals
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(romanov_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
romanov_scores[feature_index_dict] = residuals

In [59]:
K = 156
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
print(len(ind_romanov_zeisel))
filtered_data_train = data_train[ind_romanov_zeisel, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))

100


In [35]:
#PCA
data = np.hstack((romanov_logcounts_share,zeisel_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))

In [ ]:
#小鼠视网膜

In [36]:
shekhar_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_feature.csv').iloc[:,1].values.tolist()
macosko_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_feature.csv').iloc[:,1].values.tolist()
shekhar_macosko = intersection(shekhar_feature, macosko_feature)
shekhar_feature_share = [shekhar_feature.index(i) for i in shekhar_macosko]
macosko_feature_share = [macosko_feature.index(i) for i in shekhar_macosko]
shekhar_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_logcounts.csv').iloc[:,1:].values
macosko_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_logcounts.csv').iloc[:,1:].values
shekhar_logcounts_share = shekhar_logcounts[shekhar_feature_share, :]
macosko_logcounts_share = macosko_logcounts[macosko_feature_share, :]
feature_number = len(shekhar_macosko)
shekhar_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_label.csv').iloc[:,1].values.tolist()
macosko_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_label.csv').iloc[:,1].values.tolist()

In [37]:
pd.value_counts(macosko_label)

rods                    29400
bipolar                  6285
amacrine                 4426
cones                    1868
muller                   1624
ganglion                  432
horizontal                252
vascular_endothelium      252
fibroblasts                85
microglia                  67
pericytes                  63
astrocytes                 54
dtype: int64

In [38]:
pd.value_counts(shekhar_label)

bipolar     23494
muller       2945
amacrine      252
rods           91
cones          48
dtype: int64

In [48]:
test_name = 'macosko'
if test_name == 'macosko':
    data_train = shekhar_logcounts_share
    test_logcounts_share = macosko_logcounts_share
    label_test = macosko_label
    label_train = shekhar_label
if test_name == 'shekhar':
    data_train = macosko_logcounts_share
    test_logcounts_share = shekhar_logcounts_share
    label_test = shekhar_label
    label_train = macosko_label
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [49]:
#scmap
shekhar_scores = np.zeros(feature_number)-100
data_original = np.exp(shekhar_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
shekhar_scores[feature_index_dict] = residuals
print(residuals.shape)

macosko_scores = np.zeros(feature_number)-100
data_original = np.exp(macosko_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
macosko_scores[feature_index_dict] = residuals
print(residuals.shape)


(12333,)
(12332,)


In [50]:
K = 298
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_shekhar_macosko = intersection(ind_shekhar, ind_macosko)
print(len(ind_shekhar_macosko))
filtered_data_train = data_train[ind_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_shekhar_macosko, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))


100


In [51]:
#superCT
print(data_train.shape)
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))

(12333, 26830)


In [52]:
# 460147 论文方法
data_original = np.exp(shekhar_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
shekhar_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(macosko_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
macosko_scores = (data_train_var_log-y_pred).reshape(-1)


In [53]:
K = 228
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_shekhar_macosko = intersection(ind_shekhar, ind_macosko)
print(len(ind_shekhar_macosko))
filtered_data_train = data_train[ind_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_shekhar_macosko, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_460147.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_460147.hkl'%(test_name,K))


100


In [54]:
#myself 自己方法
macosko_scores = np.zeros(feature_number)-100
data_original = np.exp(macosko_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(macosko_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
macosko_scores[feature_index_dict] = residuals
shekhar_scores = np.zeros(feature_number)-100
data_original = np.exp(shekhar_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(shekhar_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
shekhar_scores[feature_index_dict] = residuals

In [55]:
K = 270
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_shekhar_macosko = intersection(ind_shekhar, ind_macosko)
print(len(ind_shekhar_macosko))
filtered_data_train = data_train[ind_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_shekhar_macosko, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))


100


In [56]:
#PCA
data = np.hstack((shekhar_logcounts_share,macosko_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))


In [3]:
#人类胰腺
muraro_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_feature.csv').iloc[:,1].values.tolist()
baron_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_feature.csv').iloc[:,1].values.tolist()
xin_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_feature.csv').iloc[:,1].values.tolist()
segerstolpe_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_feature.csv').iloc[:,1].values.tolist()

In [4]:
#找到共同基因集合
muraro_baron = intersection(muraro_feature, baron_feature)
muraro_baron_xin = intersection(muraro_baron, xin_feature)
muraro_baron_xin_segerstolpe = intersection(muraro_baron_xin, segerstolpe_feature)
#找到这些相同基因位置
muraro_feature_share = [muraro_feature.index(i) for i in muraro_baron_xin_segerstolpe]
baron_feature_share = [baron_feature.index(i) for i in muraro_baron_xin_segerstolpe]
xin_feature_share = [xin_feature.index(i) for i in muraro_baron_xin_segerstolpe]
segerstolpe_feature_share = [segerstolpe_feature.index(i) for i in muraro_baron_xin_segerstolpe]
#读取数据集
muraro_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_logcounts.csv').iloc[:,1:].values
baron_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_logcounts.csv').iloc[:,1:].values
xin_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_logcounts.csv').iloc[:,1:].values
segerstolpe_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_logcounts.csv').iloc[:,1:].values
#所用数据集（未进行降维）
muraro_logcounts_share = muraro_logcounts[muraro_feature_share, :]
baron_logcounts_share = baron_logcounts[baron_feature_share, :]
xin_logcounts_share = xin_logcounts[xin_feature_share, :]
segerstolpe_logcounts_share = segerstolpe_logcounts[segerstolpe_feature_share, :]

In [243]:
muraro_share = np.exp(muraro_logcounts_share)-1
muraro_share = pd.DataFrame(muraro_share)
muraro_share.to_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro_share.csv')
xin_share = np.exp(xin_logcounts_share)-1
xin_share = pd.DataFrame(xin_share)
xin_share.to_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin_share.csv')
baron_share = np.exp(baron_logcounts_share)-1
baron_share = pd.DataFrame(baron_share)
baron_share.to_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron_share.csv')
segerstolpe_share = np.exp(segerstolpe_logcounts_share)-1
segerstolpe_share = pd.DataFrame(segerstolpe_share)
segerstolpe_share.to_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe_share.csv')


In [5]:
feature_number = len(muraro_baron_xin_segerstolpe)

In [6]:
#muraro scmap
muraro_scores = np.zeros(feature_number)-100
data_original = np.exp(muraro_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
muraro_scores[feature_index_dict] = residuals
#xin scmap
xin_scores = np.zeros(feature_number)-100
data_original = np.exp(xin_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
xin_scores[feature_index_dict] = residuals
print(residuals.shape)
#baron scmap
baron_scores = np.zeros(feature_number)-100
data_original = np.exp(baron_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
baron_scores[feature_index_dict] = residuals
print(residuals.shape)
#segerstolpe scmap
segerstolpe_scores = np.zeros(feature_number)-100
data_original = np.exp(segerstolpe_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
segerstolpe_scores[feature_index_dict] = residuals
print(residuals.shape)

(16088,)
(15917,)
(16243,)


In [13]:
K = 1229
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
len(ind_muraro_baron_xin_segerstolpe)

100

In [6]:
test_name = 'segerstolpe'
if test_name == 'muraro':
    data_train = np.hstack((baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'baron':
    data_train = np.hstack((muraro_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'xin':
    data_train = np.hstack((muraro_logcounts_share, baron_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'segerstolpe':
    data_train = np.hstack((xin_logcounts_share, baron_logcounts_share, muraro_logcounts_share))
muraro_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_label.csv').iloc[:,1].values.tolist()
if test_name == 'muraro':
    label_train = baron_label+xin_label+segerstolpe_label
    test_logcounts_share = muraro_logcounts_share
if test_name == 'baron':
    label_train = muraro_label+xin_label+segerstolpe_label
    test_logcounts_share = baron_logcounts_share
if test_name == 'xin':
    label_train = muraro_label+baron_label+segerstolpe_label
    test_logcounts_share = xin_logcounts_share
if test_name == 'segerstolpe':
    label_train = xin_label+baron_label+muraro_label
    test_logcounts_share = segerstolpe_logcounts_share
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))

NameError: name 'ind_muraro_baron_xin_segerstolpe' is not defined

In [7]:
test_name = 'segerstolpe'
if test_name == 'muraro':
    data_train = np.hstack((baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'baron':
    data_train = np.hstack((muraro_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'xin':
    data_train = np.hstack((muraro_logcounts_share, baron_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'segerstolpe':
    data_train = np.hstack((xin_logcounts_share, baron_logcounts_share, muraro_logcounts_share))
muraro_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_label.csv').iloc[:,1].values.tolist()
if test_name == 'muraro':
    label_train = baron_label+xin_label+segerstolpe_label
    test_logcounts_share = muraro_logcounts_share
if test_name == 'baron':
    label_train = muraro_label+xin_label+segerstolpe_label
    test_logcounts_share = baron_logcounts_share
if test_name == 'xin':
    label_train = muraro_label+baron_label+segerstolpe_label
    test_logcounts_share = xin_logcounts_share
if test_name == 'segerstolpe':
    label_train = xin_label+baron_label+muraro_label
    test_logcounts_share = segerstolpe_logcounts_share
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
print(data_train.shape)
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))

(16459, 12183)


In [164]:
data_original = np.exp(baron_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
baron_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(muraro_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
muraro_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(xin_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
xin_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(segerstolpe_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
segerstolpe_scores = (data_train_var_log-y_pred).reshape(-1)

In [168]:
K = 1165
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
len(ind_muraro_baron_xin_segerstolpe)

100

In [172]:
test_name = 'segerstolpe'
if test_name == 'muraro':
    data_train = np.hstack((baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'baron':
    data_train = np.hstack((muraro_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'xin':
    data_train = np.hstack((muraro_logcounts_share, baron_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'segerstolpe':
    data_train = np.hstack((xin_logcounts_share, baron_logcounts_share, muraro_logcounts_share))
muraro_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_label.csv').iloc[:,1].values.tolist()
if test_name == 'muraro':
    label_train = baron_label+xin_label+segerstolpe_label
    test_logcounts_share = muraro_logcounts_share
if test_name == 'baron':
    label_train = muraro_label+xin_label+segerstolpe_label
    test_logcounts_share = baron_logcounts_share
if test_name == 'xin':
    label_train = muraro_label+baron_label+segerstolpe_label
    test_logcounts_share = xin_logcounts_share
if test_name == 'segerstolpe':
    label_train = xin_label+baron_label+muraro_label
    test_logcounts_share = segerstolpe_logcounts_share
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_460147.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_460147.hkl'%(test_name,K))

In [178]:
test_name = 'muraro'
if test_name == 'muraro':
    data_train = np.hstack((baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'baron':
    data_train = np.hstack((muraro_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'xin':
    data_train = np.hstack((muraro_logcounts_share, baron_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'segerstolpe':
    data_train = np.hstack((xin_logcounts_share, baron_logcounts_share, muraro_logcounts_share))
if test_name == 'muraro':
    label_train = baron_label+xin_label+segerstolpe_label
    test_logcounts_share = muraro_logcounts_share
if test_name == 'baron':
    label_train = muraro_label+xin_label+segerstolpe_label
    test_logcounts_share = baron_logcounts_share
if test_name == 'xin':
    label_train = muraro_label+baron_label+segerstolpe_label
    test_logcounts_share = xin_logcounts_share
if test_name == 'segerstolpe':
    label_train = xin_label+baron_label+muraro_label
    test_logcounts_share = segerstolpe_logcounts_share
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)

In [179]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
data = np.hstack((muraro_logcounts_share, baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))

In [14]:
#myself
muraro_scores = np.zeros(feature_number)-100
data_original = np.exp(muraro_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(muraro_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
muraro_scores[feature_index_dict] = residuals
baron_scores = np.zeros(feature_number)-100
data_original = np.exp(baron_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(baron_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
baron_scores[feature_index_dict] = residuals
xin_scores = np.zeros(feature_number)-100
data_original = np.exp(xin_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(xin_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
xin_scores[feature_index_dict] = residuals
segerstolpe_scores = np.zeros(feature_number)-100
data_original = np.exp(segerstolpe_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(segerstolpe_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
segerstolpe_scores[feature_index_dict] = residuals

In [82]:
K = 3260
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
len(ind_muraro_baron_xin_segerstolpe)

300

In [24]:
muraro_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_label.csv').iloc[:,1].values.tolist()

In [83]:
test_name = 'segerstolpe'
if test_name == 'muraro':
    data_train = np.hstack((baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'baron':
    data_train = np.hstack((muraro_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'xin':
    data_train = np.hstack((muraro_logcounts_share, baron_logcounts_share, segerstolpe_logcounts_share))
if test_name == 'segerstolpe':
    data_train = np.hstack((xin_logcounts_share, baron_logcounts_share, muraro_logcounts_share))
if test_name == 'muraro':
    label_train = baron_label+xin_label+segerstolpe_label
    test_logcounts_share = muraro_logcounts_share
if test_name == 'baron':
    label_train = muraro_label+xin_label+segerstolpe_label
    test_logcounts_share = baron_logcounts_share
if test_name == 'xin':
    label_train = muraro_label+baron_label+segerstolpe_label
    test_logcounts_share = xin_logcounts_share
if test_name == 'segerstolpe':
    label_train = xin_label+baron_label+muraro_label
    test_logcounts_share = segerstolpe_logcounts_share
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe, :]
K = len(ind_muraro_baron_xin_segerstolpe)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))

In [ ]:
#同类异源
#小鼠视网膜=>小鼠脑细胞
#小鼠脑细胞=>小鼠视网膜

In [100]:
shekhar_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_feature.csv').iloc[:,1].values.tolist()
macosko_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_feature.csv').iloc[:,1].values.tolist()
romanov_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_feature.csv').iloc[:,1].values.tolist()
zeisel_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_feature.csv').iloc[:,1].values.tolist()

shekhar_macosko = intersection(shekhar_feature, macosko_feature)
shekhar_macosko_romanov = intersection(shekhar_macosko, romanov_feature)
shekhar_macosko_romanov_zeisel = intersection(shekhar_macosko_romanov, zeisel_feature)
shekhar_feature_share = [shekhar_feature.index(i) for i in shekhar_macosko_romanov_zeisel]
macosko_feature_share = [macosko_feature.index(i) for i in shekhar_macosko_romanov_zeisel]
romanov_feature_share = [romanov_feature.index(i) for i in shekhar_macosko_romanov_zeisel]
zeisel_feature_share = [zeisel_feature.index(i) for i in shekhar_macosko_romanov_zeisel]

shekhar_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_logcounts.csv').iloc[:,1:].values
macosko_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_logcounts.csv').iloc[:,1:].values
romanov_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_logcounts.csv').iloc[:,1:].values
zeisel_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_logcounts.csv').iloc[:,1:].values

shekhar_logcounts_share = shekhar_logcounts[shekhar_feature_share, :]
macosko_logcounts_share = macosko_logcounts[macosko_feature_share, :]
romanov_logcounts_share = romanov_logcounts[romanov_feature_share, :]
zeisel_logcounts_share = zeisel_logcounts[zeisel_feature_share, :]

feature_number = len(shekhar_macosko_romanov_zeisel)
shekhar_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/shekhar/shekhar_label.csv').iloc[:,1].values.tolist()
macosko_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/macosko/macosko_label.csv').iloc[:,1].values.tolist()
romanov_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/romanov/romanov_label.csv').iloc[:,1].values.tolist()
zeisel_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/zeisel/zeisel_label.csv').iloc[:,1].values.tolist()

In [101]:
for i in range(len(romanov_label)):
    if romanov_label[i] == 'oligos':
        romanov_label[i] = 'oligodendrocytes'
for i in range(len(zeisel_label)):
    if zeisel_label[i] == 'ca1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 's1pyramidal':
        zeisel_label[i] = 'neurons'
    if zeisel_label[i] == 'interneurons':
        zeisel_label[i] = 'neurons'

In [186]:
data_train = np.hstack((romanov_logcounts_share,zeisel_logcounts_share))
label_train = romanov_label+zeisel_label
test_logcounts_share = np.hstack((shekhar_logcounts_share,macosko_logcounts_share))
label_test = shekhar_label+macosko_label
test_name = "retina"
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)
test_name = "brain"
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [187]:
test_name = "retina"
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))
test_name = "brain"
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))


In [103]:
#scmap
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
romanov_scores[feature_index_dict] = residuals
print(residuals.shape)

zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
zeisel_scores[feature_index_dict] = residuals
print(residuals.shape)
shekhar_scores = np.zeros(feature_number)-100
data_original = np.exp(shekhar_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
shekhar_scores[feature_index_dict] = residuals
print(residuals.shape)

macosko_scores = np.zeros(feature_number)-100
data_original = np.exp(macosko_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
macosko_scores[feature_index_dict] = residuals
print(residuals.shape)

(11726,)
(11732,)
(11732,)
(11732,)


In [109]:
K = 1140
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_shekhar = intersection(ind_romanov_zeisel, ind_shekhar)
ind_romanov_zeisel_shekhar_macosko = intersection(ind_romanov_zeisel_shekhar, ind_macosko)
print(len(ind_romanov_zeisel_shekhar_macosko))
filtered_data_train = data_train[ind_romanov_zeisel_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_shekhar_macosko, :]
K = 100
test_name = "retina"
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))
test_name = "brain"
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))

100


In [110]:
data = np.hstack((romanov_logcounts_share,zeisel_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
test_name = "retina"
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))
test_name = "brain"
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))

In [126]:
#myself 自己方法
zeisel_scores = np.zeros(feature_number)-100
data_original = np.exp(zeisel_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(zeisel_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
zeisel_scores[feature_index_dict] = residuals
romanov_scores = np.zeros(feature_number)-100
data_original = np.exp(romanov_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(romanov_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
romanov_scores[feature_index_dict] = residuals
macosko_scores = np.zeros(feature_number)-100
data_original = np.exp(macosko_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(macosko_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
macosko_scores[feature_index_dict] = residuals
shekhar_scores = np.zeros(feature_number)-100
data_original = np.exp(shekhar_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(shekhar_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
shekhar_scores[feature_index_dict] = residuals

In [133]:
K = 990
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_shekhar = intersection(ind_romanov_zeisel, ind_shekhar)
ind_romanov_zeisel_shekhar_macosko = intersection(ind_romanov_zeisel_shekhar, ind_macosko)
print(len(ind_romanov_zeisel_shekhar_macosko))
filtered_data_train = data_train[ind_romanov_zeisel_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_shekhar_macosko, :]
K = 100
test_name = "retina"
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))
test_name = "brain"
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))

100


In [134]:
data_original = np.exp(romanov_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
romanov_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(zeisel_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
zeisel_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(shekhar_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
shekhar_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(macosko_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
macosko_scores = (data_train_var_log-y_pred).reshape(-1)

In [142]:
K = 1450
ind_romanov = np.argpartition(romanov_scores, -K)[-K:].tolist()
ind_zeisel = np.argpartition(zeisel_scores, -K)[-K:].tolist()
ind_shekhar = np.argpartition(shekhar_scores, -K)[-K:].tolist()
ind_macosko = np.argpartition(macosko_scores, -K)[-K:].tolist()
ind_romanov_zeisel = intersection(ind_romanov, ind_zeisel)
ind_romanov_zeisel_shekhar = intersection(ind_romanov_zeisel, ind_shekhar)
ind_romanov_zeisel_shekhar_macosko = intersection(ind_romanov_zeisel_shekhar, ind_macosko)
print(len(ind_romanov_zeisel_shekhar_macosko))
filtered_data_train = data_train[ind_romanov_zeisel_shekhar_macosko, :]
filtered_data_test = test_logcounts_share[ind_romanov_zeisel_shekhar_macosko, :]
K = 100
test_name = "retina"
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))
test_name = "brain"
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))

100


In [269]:
#baron2
muraro_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_feature.csv').iloc[:,1].values.tolist()
baron_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_feature.csv').iloc[:,1].values.tolist()
xin_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_feature.csv').iloc[:,1].values.tolist()
segerstolpe_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_feature.csv').iloc[:,1].values.tolist()
baron2_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron2/baron2_feature.csv').iloc[:,1].values.tolist()

In [ ]:
baron2_feature = [i.upper() for  i in baron2_feature]

In [252]:
#baron2专用
test_name = 'baron2'
#找到共同基因集合
muraro_baron = intersection(muraro_feature, baron_feature)
muraro_baron_xin = intersection(muraro_baron, xin_feature)
muraro_baron_xin_segerstolpe = intersection(muraro_baron_xin, segerstolpe_feature)
muraro_baron_xin_segerstolpe_baron2 = intersection(muraro_baron_xin_segerstolpe, baron2_feature)
#找到这些相同基因位置
muraro_feature_share = [muraro_feature.index(i) for i in muraro_baron_xin_segerstolpe_baron2]
baron_feature_share = [baron_feature.index(i) for i in muraro_baron_xin_segerstolpe_baron2]
xin_feature_share = [xin_feature.index(i) for i in muraro_baron_xin_segerstolpe_baron2]
segerstolpe_feature_share = [segerstolpe_feature.index(i) for i in muraro_baron_xin_segerstolpe_baron2]
baron2_feature_share = [baron2_feature.index(i) for i in muraro_baron_xin_segerstolpe_baron2]


In [254]:
#读取数据集
muraro_logcounts = pd.read_csv('/home/chenshengquan/data/scmap/muraro_logcounts.csv').iloc[:,1:].values
baron_logcounts = pd.read_csv('/home/chenshengquan/data/scmap/baron_logcounts.csv').iloc[:,1:].values
xin_logcounts = pd.read_csv('/home/chenshengquan/data/scmap/xin_logcounts.csv').iloc[:,1:].values
segerstolpe_logcounts = pd.read_csv('/home/chenshengquan/data/scmap/segerstolpe_logcounts.csv').iloc[:,1:].values
baron2_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron2/baron2_logcounts.csv').iloc[:,1:].values
#所用数据集（未进行降维）
muraro_logcounts_share = muraro_logcounts[muraro_feature_share, :]
baron_logcounts_share = baron_logcounts[baron_feature_share, :]
xin_logcounts_share = xin_logcounts[xin_feature_share, :]
segerstolpe_logcounts_share = segerstolpe_logcounts[segerstolpe_feature_share, :]
baron2_logcounts_share = baron2_logcounts[baron2_feature_share,:]


In [255]:
muraro_label = pd.read_csv('/home/chenshengquan/data/scmap/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenshengquan/data/scmap/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenshengquan/data/scmap/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenshengquan/data/scmap/segerstolpe_label.csv').iloc[:,1].values.tolist()
baron2_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron2/baron2_label.csv').iloc[:,1].values.tolist()
label_train = muraro_label+baron_label+xin_label+segerstolpe_label
label_test = baron2_label
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [256]:
data_train = np.hstack((muraro_logcounts_share,baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
test_logcounts_share = baron2_logcounts_share


In [276]:
pd.value_counts(baron2_label)

beta                  894
ductal                275
delta                 218
alpha                 191
endothelial           139
quiescent_stellate     47
gamma                  41
macrophage             36
activated_stellate     14
B_cell                 10
immune_other            8
T_cell                  7
schwann                 6
dtype: int64

In [273]:
A = []
k = 0
for i in range(len(baron2_label)):
    if baron2_label[i] in label_train:
        k = k+1


In [275]:
k/len(baron2_label)

0.986744432661718

In [253]:
len(muraro_baron_xin_segerstolpe_baron2)

11948

In [257]:
muraro_scores = np.zeros(feature_number)-100
data_original = np.exp(muraro_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(muraro_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
muraro_scores[feature_index_dict] = residuals
baron_scores = np.zeros(feature_number)-100
data_original = np.exp(baron_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(baron_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
baron_scores[feature_index_dict] = residuals
xin_scores = np.zeros(feature_number)-100
data_original = np.exp(xin_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(xin_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
xin_scores[feature_index_dict] = residuals
segerstolpe_scores = np.zeros(feature_number)-100
data_original = np.exp(segerstolpe_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(segerstolpe_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
segerstolpe_scores[feature_index_dict] = residuals
baron2_scores = np.zeros(feature_number)-100
data_original = np.exp(baron2_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(baron2_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
baron2_scores[feature_index_dict] = residuals



In [264]:
K = 1945
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()
ind_baron2 = np.argpartition(baron2_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
ind_muraro_baron_xin_segerstolpe_baron2 =  intersection(ind_muraro_baron_xin_segerstolpe,ind_baron2)
print(len(ind_muraro_baron_xin_segerstolpe_baron2))
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe_baron2, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe_baron2, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))


100


In [266]:
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))

In [267]:
pd.value_counts(label_train)

alpha                     4910
beta                      3715
ductal                    1708
acinar                    1362
delta                      957
gamma                      638
endothelial                289
activated_stellate         284
quiescent_stellate         173
mesenchymal                 80
macrophage                  55
PSC                         54
unclassified endocrine      41
co-expression               39
mast                        32
epsilon                     28
schwann                     13
t_cell                       7
MHC class II                 5
unclear                      4
unclassified                 2
dtype: int64

In [ ]:
#人类胰腺=>人类脑细胞

In [244]:
muraro_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_feature.csv').iloc[:,1].values.tolist()
baron_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_feature.csv').iloc[:,1].values.tolist()
xin_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_feature.csv').iloc[:,1].values.tolist()
segerstolpe_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_feature.csv').iloc[:,1].values.tolist()
darmanis_feature = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_feature.csv').iloc[:,1].values.tolist()

In [189]:
#darmanis专用
test_name = 'darmanis'
#找到共同基因集合
muraro_baron = intersection(muraro_feature, baron_feature)
muraro_baron_xin = intersection(muraro_baron, xin_feature)
muraro_baron_xin_segerstolpe = intersection(muraro_baron_xin, segerstolpe_feature)
muraro_baron_xin_segerstolpe_darmanis = intersection(muraro_baron_xin_segerstolpe, darmanis_feature)
#找到这些相同基因位置
muraro_feature_share = [muraro_feature.index(i) for i in muraro_baron_xin_segerstolpe_darmanis]
baron_feature_share = [baron_feature.index(i) for i in muraro_baron_xin_segerstolpe_darmanis]
xin_feature_share = [xin_feature.index(i) for i in muraro_baron_xin_segerstolpe_darmanis]
segerstolpe_feature_share = [segerstolpe_feature.index(i) for i in muraro_baron_xin_segerstolpe_darmanis]
darmanis_feature_share = [darmanis_feature.index(i) for i in muraro_baron_xin_segerstolpe_darmanis]
#读取数据集
muraro_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_logcounts.csv').iloc[:,1:].values
baron_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_logcounts.csv').iloc[:,1:].values
xin_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_logcounts.csv').iloc[:,1:].values
segerstolpe_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_logcounts.csv').iloc[:,1:].values
darmanis_logcounts = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_logcounts.csv').iloc[:,1:].values
#所用数据集（未进行降维）
muraro_logcounts_share = muraro_logcounts[muraro_feature_share, :]
baron_logcounts_share = baron_logcounts[baron_feature_share, :]
xin_logcounts_share = xin_logcounts[xin_feature_share, :]
segerstolpe_logcounts_share = segerstolpe_logcounts[segerstolpe_feature_share, :]
darmanis_logcounts_share = darmanis_logcounts[darmanis_feature_share,:]

In [190]:
len(muraro_baron_xin_segerstolpe_darmanis)

15955

In [192]:
if not os.path.exists('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name):
    os.makedirs('/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual'%test_name)

In [193]:
muraro_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/muraro/muraro_label.csv').iloc[:,1].values.tolist()
baron_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/baron/baron_label.csv').iloc[:,1].values.tolist()
xin_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/xin/xin_label.csv').iloc[:,1].values.tolist()
segerstolpe_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/segerstolpe/segerstolpe_label.csv').iloc[:,1].values.tolist()
darmanis_label = pd.read_csv('/home/chenxiaoyang/program/scmap/Data/processed/darmanis/darmanis_label.csv').iloc[:,1].values.tolist()
label_train = muraro_label+baron_label+xin_label+segerstolpe_label
label_test = darmanis_label
hkl.dump(label_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_train_label.hkl'%test_name)
hkl.dump(label_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/sorted_residual/data_test_label.hkl'%test_name)

In [194]:
data_train = np.hstack((muraro_logcounts_share,baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
test_logcounts_share = darmanis_logcounts_share

In [195]:
muraro_scores = np.zeros(feature_number)-100
data_original = np.exp(muraro_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(muraro_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
muraro_scores[feature_index_dict] = residuals
baron_scores = np.zeros(feature_number)-100
data_original = np.exp(baron_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(baron_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
baron_scores[feature_index_dict] = residuals
xin_scores = np.zeros(feature_number)-100
data_original = np.exp(xin_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(xin_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
xin_scores[feature_index_dict] = residuals
segerstolpe_scores = np.zeros(feature_number)-100
data_original = np.exp(segerstolpe_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(segerstolpe_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
segerstolpe_scores[feature_index_dict] = residuals
darmanis_scores = np.zeros(feature_number)-100
data_original = np.exp(darmanis_logcounts_share) - 1
drop_feature = []
X1 = []
Y = []
X2 = []
feature_index_dict = []
#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
data_train_mean = np.mean(data_original,1)
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        Y.append(np.log(data_train_mean[i]+1))
        X1.append(np.mean(darmanis_logcounts_share[i,:]))
        X2.append(np.log(drop_feature[i]))
Y = np.array(Y).reshape(-1,1)
X1 = np.array(X1).reshape(-1,1)
X2 = np.array(X2).reshape(-1,1)
lr = LinearRegression()
lr.fit(X2,Y)
predictions = lr.predict(X2)
residuals = (Y-predictions).reshape(-1) + (Y - X1).reshape(-1)
darmanis_scores[feature_index_dict] = residuals


In [201]:
K = 2600
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
ind_muraro_baron_xin_segerstolpe_darmanis =  intersection(ind_muraro_baron_xin_segerstolpe,ind_darmanis)
print(len(ind_muraro_baron_xin_segerstolpe_darmanis))
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe_darmanis, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_myself.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_myself.hkl'%(test_name,K))


100


In [202]:
data_original = np.exp(baron_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
baron_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(muraro_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
baron_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(xin_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
xin_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(segerstolpe_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
segerstolpe_scores = (data_train_var_log-y_pred).reshape(-1)
data_original = np.exp(darmanis_logcounts_share) - 1
data_train_mean = np.mean(data_original,1)
data_train_var = np.var(data_original,1)
data_train_mean_log = np.log(data_train_mean+1)
data_train_var_log = np.log(data_train_var+1)
ploy_reg = PolynomialFeatures(degree=2)
x_ = ploy_reg.fit_transform(data_train_mean_log.reshape(-1,1))
regr2 = linear_model.LinearRegression()
regr2.fit(x_,data_train_var_log)
y_pred = regr2.predict(x_)
darmanis_scores = (data_train_var_log-y_pred).reshape(-1)

In [209]:
K = 3390
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
ind_muraro_baron_xin_segerstolpe_darmanis =  intersection(ind_muraro_baron_xin_segerstolpe,ind_darmanis)
print(len(ind_muraro_baron_xin_segerstolpe_darmanis))
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe_darmanis, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_460147.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_460147.hkl'%(test_name,K))


100


In [210]:
#PCA
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
data = np.hstack((muraro_logcounts_share, baron_logcounts_share, xin_logcounts_share, segerstolpe_logcounts_share))
min_max_scaler = preprocessing.MinMaxScaler()
X_ = min_max_scaler.fit_transform(data.T)
X_train = min_max_scaler.transform(data_train.T)
X_test = min_max_scaler.transform(test_logcounts_share.T)
pca=PCA(n_components=100)
pca.fit(X_)
PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
hkl.dump(PCA_X_train, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_PCA.hkl'%(test_name,100))
hkl.dump(PCA_X_test, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_PCA.hkl'%(test_name,100))

In [211]:
#muraro scmap
muraro_scores = np.zeros(feature_number)-100
data_original = np.exp(muraro_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
muraro_scores[feature_index_dict] = residuals
#xin scmap
xin_scores = np.zeros(feature_number)-100
data_original = np.exp(xin_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
xin_scores[feature_index_dict] = residuals
print(residuals.shape)
#baron scmap
baron_scores = np.zeros(feature_number)-100
data_original = np.exp(baron_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
baron_scores[feature_index_dict] = residuals
print(residuals.shape)
#segerstolpe scmap
segerstolpe_scores = np.zeros(feature_number)-100
data_original = np.exp(segerstolpe_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
segerstolpe_scores[feature_index_dict] = residuals
print(residuals.shape)
darmanis_scores = np.zeros(feature_number)-100
data_original = np.exp(darmanis_logcounts_share) - 1
data_original_mean = np.mean(data_original,1)
drop_feature = []

#寻找每个feature的0的数目（概率）
for feature in data_original:
    drop_feature.append(np.sum(feature==0)/feature.shape[0])
#print(drop_feature)
feature_index_dict = []
train_E = []
train_D = []
for i in range(len(drop_feature)):
    if (drop_feature[i]!=1 and drop_feature[i]!=0):
        feature_index_dict.append(i)
        train_E.append(data_original_mean[i])
        train_D.append(drop_feature[i])
train_E = np.array(train_E)
train_D = np.array(train_D)
train_E = np.log(train_E+1).reshape(-1,1)*math.log(2.7)/math.log(2)
train_D = np.log(100*train_D).reshape(-1,1)*math.log(2.7)/math.log(2)

lr = LinearRegression()
lr.fit(train_E,train_D)
predictions = lr.predict(train_E)
residuals = (train_D-predictions).reshape(-1)
#residuals = np.abs(residuals)
darmanis_scores[feature_index_dict] = residuals
print(residuals.shape)

(15603,)
(15451,)
(15781,)
(15802,)


In [220]:
K = 2630
ind_muraro = np.argpartition(muraro_scores, -K)[-K:].tolist()
ind_baron = np.argpartition(baron_scores, -K)[-K:].tolist()
ind_xin = np.argpartition(xin_scores, -K)[-K:].tolist()
ind_segerstolpe = np.argpartition(segerstolpe_scores, -K)[-K:].tolist()
ind_darmanis = np.argpartition(darmanis_scores, -K)[-K:].tolist()

ind_muraro_baron = intersection(ind_muraro, ind_baron)
ind_muraro_baron_xin = intersection(ind_muraro_baron, ind_xin)
ind_muraro_baron_xin_segerstolpe = intersection(ind_muraro_baron_xin, ind_segerstolpe)
ind_muraro_baron_xin_segerstolpe_darmanis =  intersection(ind_muraro_baron_xin_segerstolpe,ind_darmanis)
print(len(ind_muraro_baron_xin_segerstolpe_darmanis))
filtered_data_train = data_train[ind_muraro_baron_xin_segerstolpe_darmanis, :]
filtered_data_test = test_logcounts_share[ind_muraro_baron_xin_segerstolpe_darmanis, :]
K = 100
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_scmap.hkl'%(test_name,K))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_scmap.hkl'%(test_name,K))


100


In [221]:
filtered_data_train = np.int64(data_train>0)
filtered_data_test = np.int64(test_logcounts_share>0)
hkl.dump(filtered_data_train.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_train_%d_sample_SuperCT.hkl'%(test_name,100))
hkl.dump(filtered_data_test.T, '/home/chenxiaoyang/program/scmap/Data/processed/%s/data_test_%d_sample_SuperCT.hkl'%(test_name,100))